In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import pyodbc
from datetime import datetime
from selenium.common.exceptions import NoSuchElementException
from selenium.common import exceptions

In [ ]:
instanteInicial = datetime.now()
#############################################

##CONFIGURACION DRIVER Y URL BASE

driver = webdriver.Firefox(executable_path = "./Firefox/geckodriver.exe")
lista_direcciones =  ['Av. Bicentenario 3236, Vitacura '  , 'Av San Josemaría Escrivá de Balaguer 6400, Vitacura', 'Vía Verde 3850, Vitacura', 'Buenaventura 1770, Vitacura', 'El Manantial 1781, Santiago, Vitacura']
nombre_restaurant = []
url_restaurant = []
driver.get("https://www.rappi.cl/restaurantes")

for direccion in lista_direcciones:
    time.sleep(5)
    titulo= driver.title
    cantidad_restaurantes = re.findall(r'-?\d+\.?\d*', titulo)
    cantidad_restaurantes  = cantidad_restaurantes [0]
    cantidad_restaurantes  = int(cantidad_restaurantes)
    iteracion=cantidad_restaurantes/25
    iteracion = int(iteracion)

    time.sleep(5)

    button_1 = driver.find_element(By.XPATH, '//div[@class="sc-dwLEzm hIJdYR ButtonAddress__text"]')
    button_1.click()

    time.sleep(5)

    search_bar = driver.find_element(By.XPATH, '//input[@class="chakra-input css-5p5pfl"]')
    search_bar.send_keys(direccion)
    time.sleep(5)

    button_2 = driver.find_element(By.XPATH, '//button[@class="chakra-button css-1p0d4h7"]')
    button_2.click()
    time.sleep(5)
    button_3 = driver.find_element(By.XPATH, '//button[@class="chakra-button css-rbd1d8"]')
    button_3.click()

    time.sleep(5)

    button_4 = driver.find_element(By.XPATH, '//button[@class="chakra-button css-rbd1d8"]')
    button_4.click()

    time.sleep(5)

        
    for i in range(0,iteracion): 
        try:
            time.sleep(3)
            button = driver.find_element(By.XPATH, '//button[@class="sc-hqyNC sc-jbKcbu bvSdOe primary wide"]')
            button.click()
        
        except NoSuchElementException :
            ("No quedan botones")
            

    #time.sleep(5)
    ##OBTENCION DE TITULOS
    title_products = driver.find_elements(by=By.XPATH, value='//h3[@class="sc-bxivhb bLhELA sc-946fb2bd-3 eqtXRP"]')
    for title in title_products:
        titulo_restaurant = title.text
        nombre_restaurant.append(titulo_restaurant)

    links_products = driver.find_elements(by=By.XPATH, value='//a[@class="sc-946fb2bd-2 iNTDxV"]')

    for link in links_products:
        link_restaurant = link.get_attribute("href")
        url_restaurant.append(link_restaurant)
    time.sleep(5)
    
    driver.refresh()

driver.close()


#############################################
instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds
print(tiempo,segundos," Tiempo Consulta")

C:\Users\adm\AppData\Local\Temp\ipykernel_14340\1818322476.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = "./Firefox/geckodriver.exe")


In [23]:
##CREACION DICCIONARIO RESPALDO DE DATOS
data_products ={"Nombre Restaurant":nombre_restaurant,
               "URL RESTAURANT":url_restaurant }

In [26]:
##LIMPIEZA DE DUPLICADOS URL
links_products = []
for url in url_restaurant:
    if url not in links_products:
        links_products.append(url)

In [27]:
##LIMPIEZA DE DUPLICADOS TITULO RESTAURANT
title_products = []
for title in nombre_restaurant:
    if title not in title_products:
        title_products.append(title)

# SCRAPEO DE MENUS POR RESTAURANT (BEAUTIFULSOUP)

In [40]:
instanteInicial = datetime.now()
#############################################


##AGENTE PARA SCRAPEO
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept-Language': 'en-US, en;q=0.5'
}

##CREACION DE LISTAS PARA GUARADAR DATOS
url = []
platos = []
descripcion = []
nombre = []
direccion = []

contador_plato = 0
##CREACION CICLO FOR PARA ITERAR LAS URLS OBTENIDAS ANTERIORMENTE
for link in links_products:
    
    time.sleep(1)
    base_url = (link)
    response = requests.get(base_url, headers=headers)
    soup = BeautifulSoup(response.content , 'html.parser')
    soup_plato = soup.find_all('div',class_='css-cxew8w')
    
    ##CREACION DE CICLO FOR PARA AÑADIR PLATOS DESDE LAS URLS ANTERIORMENTE ENTREGADAS

    for soup_platos in soup_plato:
        titulo_menu = soup_platos.find('p',class_='chakra-text css-puxjan').text.strip()
        descripcion_menu = soup_platos.find('p',class_='chakra-text sc-a04fe063-2 gHQcCO css-1rmjo0r').text.strip()
        restaurant_nombre = soup.find('h1', class_='chakra-text css-6oec1p').text.strip()
        restaurant_direccion = soup.find('h2', class_='sc-bxivhb dVvqfA').text.strip()
        platos.append(titulo_menu)
        descripcion.append(descripcion_menu)
        nombre.append(restaurant_nombre)
        direccion.append(restaurant_direccion)
        url.append(link)
        lista_platos = [nombre,direccion,platos,descripcion,url]
        print("SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT: ", restaurant_nombre)

        
#############################################
instanteFinal = datetime.now()
tiempo = instanteFinal - instanteInicial # Devuelve un objeto timedelta
segundos = tiempo.seconds
print(tiempo,segundos," Tiempo Consulta")

SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Niu Sushi
SE AGREGO EL PLATO CORRECTAMENTE DEL RES

SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Chicken Factory
SE AGREGO 

SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pronto Copec
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Pront

SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREGO EL PLATO CORRECTAMENTE DEL RESTAURANT:  Roof Burger
SE AGREG

KeyboardInterrupt: 

In [41]:
lista_platos

[['Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sushi',
  'Niu Sus

# CREACION PRIMER DATAFRAME

In [43]:
df_menu = pd.DataFrame(lista_platos)
##TRANSPONER DATAFRAME (INVERSION DE COLUMNAS X FILAS)
df_menu = df_menu.transpose()
df_menu.columns = ['RESTAURANT','DIRECCION','NOMBRE_PLATO','DESCRIPCION_PLATO','URL']
df_menu.to_csv("CHICUREO_test_SIN_TELEFONO.csv", index=False , sep = ';')